In [100]:
!pip install ortools
!pip install folium
!pip install imageio
!pip install tqdm

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 78 kB 3.9 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd

In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import networkx as nx

# Creating a Vehicle Routing Model to Visit All Nodes

In [3]:
distance_matrix_df = pd.read_csv('intermediate_dataset/distance_time_matrix.csv').sort_values(['composite_gtfs_id','next_composite_gtfs_id'])
composite_stops_df = pd.read_csv('intermediate_dataset/composite_stop_names.csv')[['composite_gtfs_stops_id','stop_name','stop_lat','stop_lon']].drop_duplicates()

In [4]:
distance_matrix = distance_matrix_df.sort_values(['composite_gtfs_id','next_composite_gtfs_id'])['seconds'].to_numpy().reshape(424,424).astype(int)

In [5]:
index_to_gtfs_id = distance_matrix_df['composite_gtfs_id'].unique().tolist()

In [44]:
def get_stop_name(stop_id):
    return composite_stops_df.loc[composite_stops_df['composite_gtfs_stops_id'] == stop_id]['stop_name'].iloc[0]

In [7]:
composite_stops_df = composite_stops_df.groupby(['composite_gtfs_stops_id','stop_name'], as_index=False).agg(
    stop_lat = ('stop_lat','mean'),
    stop_lon = ('stop_lon','mean')
)

In [8]:
def create_data_model():
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data 

In [9]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)
routing = pywrapcp.RoutingModel(manager)

In [10]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [11]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        visit_list = []
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            visit_list.append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        
        
        print(plan_output)
    return total_seconds, plan_output, visit_list
    

In [12]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)
solution = routing.SolveWithParameters(search_parameters)

In [13]:
total_time, plans, visit_list = print_solution(data, manager, routing, solution)

Route for vehicle 0:
 0 ->  2 ->  4 ->  5 ->  6 ->  7 ->  8 ->  175 ->  174 ->  173 ->  172 ->  171 ->  9 ->  10 ->  11 ->  12 ->  13 ->  14 ->  15 ->  16 ->  61 ->  60 ->  59 ->  58 ->  92 ->  91 ->  105 ->  129 ->  128 ->  127 ->  126 ->  125 ->  123 ->  122 ->  121 ->  120 ->  118 ->  114 ->  113 ->  115 ->  116 ->  117 ->  119 ->  124 ->  130 ->  136 ->  140 ->  141 ->  142 ->  143 ->  144 ->  29 ->  191 ->  190 ->  189 ->  23 ->  188 ->  25 ->  26 ->  27 ->  28 ->  30 ->  31 ->  32 ->  33 ->  34 ->  35 ->  36 ->  37 ->  410 ->  62 ->  409 ->  194 ->  193 ->  192 ->  247 ->  246 ->  405 ->  406 ->  407 ->  408 ->  149 ->  150 ->  63 ->  107 ->  106 ->  384 ->  64 ->  65 ->  66 ->  67 ->  68 ->  249 ->  69 ->  70 ->  71 ->  72 ->  85 ->  86 ->  87 ->  88 ->  89 ->  90 ->  84 ->  83 ->  82 ->  81 ->  74 ->  76 ->  80 ->  79 ->  78 ->  77 ->  75 ->  73 ->  423 ->  250 ->  251 ->  252 ->  253 ->  254 ->  255 ->  256 ->  257 ->  258 ->  259 ->  260 ->  261 ->  262 ->  263 ->  264 ->  26

# Travelling Salesperson Solution

In [14]:
gtfs_id_seq = list(map(lambda x: index_to_gtfs_id[x], visit_list))

In [94]:
edge_transit

,composite_gtfs_stops_id,next_composite_gtfs_stops_id,mean_seconds_in_transit
0,101,103,100.765550
1,103,101,83.811659
2,103,104,90.000000
3,104,103,196.143498
4,104,106,90.000000
...,...,...,...
1735,R44,R43,129.060403
1736,R44,R45,120.000000
1737,R45,R44,115.570470
1738,S03,239_S04,90.000000


In [95]:
edge_transit = pd.read_csv('intermediate_dataset/edge_transit.csv')

weighted_edges = edge_transit[['composite_gtfs_stops_id','next_composite_gtfs_stops_id','mean_seconds_in_transit']].to_dict(orient='records')

weighted_edges = list(map(lambda x: (x['composite_gtfs_stops_id'], x['next_composite_gtfs_stops_id'], x['mean_seconds_in_transit']), weighted_edges))

In [16]:
G = nx.DiGraph()

G.add_weighted_edges_from(weighted_edges)

In [17]:
def get_lat_long(stop_id):
    stop_info = composite_stops_df.loc[composite_stops_df['composite_gtfs_stops_id'] == stop_id]
    assert stop_info.shape[0] > 0
    return list(stop_info[['stop_lon','stop_lat']].values[0])

lat_long = {k:get_lat_long(k) for k,v in G.nodes(data=True)}

In [18]:
import matplotlib.pyplot as plt
import matplotlib.animation

In [114]:
import tqdm
fig, ax = plt.subplots(figsize=(15,15))
for i in tqdm.tqdm(range(len(gtfs_id_seq))):
    # def update(num):
    ax.clear()
    nx.draw(G, pos=lat_long, 
        node_size=50, 
        node_color='black',
        edge_color='black',
        arrows=False,
        ax=ax
    )
    
    all_prior_nodes = gtfs_id_seq[:i+1]
    latest_station_id = all_prior_nodes[-1]
    latest_station_name = get_stop_name(latest_station_id)
    query_nodes = nx.draw(G, pos=lat_long, width=0.10, node_size=50, nodelist=all_prior_nodes, node_color='red', edge_color='black', ax=ax)
    ax.set_title(f"Frame {i+1}: {latest_station_name}", fontweight="bold")
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(f'intermediate_dataset/tsp_images/img_{i}.png', format='png')

<IPython.core.display.Javascript object>

100%|██████████| 424/424 [07:51<00:00,  1.11s/it]


In [115]:
import imageio
import glob

images = glob.glob('intermediate_dataset/tsp_images/*.png')
sorted_indices = np.argsort([int(img.split('_')[3].split('.')[0]) for img in images])
sorted_images = list(map(lambda x: images[x], sorted_indices))

ims = [imageio.imread(f) for f in sorted_images]
imageio.mimwrite('v1.gif', ims, fps=2)

/tmp/ipykernel_6460/631119851.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  ims = [imageio.imread(f) for f in sorted_images]
